A user has requested how many seven day prescriptions there are for Aspirin as a proprotion of all prescriptions for Aspirin. The aim is to use as a surrogate for dosette boxes

Aspirin 75mg is once daily so prescriptions with a seven day course length are indicative of weeekly scripts which are sometimes used (erroneously)  to support dispensing in dosette boxes. 

In [10]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [11]:
sql = '''
SELECT
  PCO_Name,
  PCO_Code,
  bnf_code AS bnf_code,
  bnf_description AS bnf_name,
  Items,
  Quantity
FROM
  ebmdatalab.tmp_eu.raw_prescribing_data_2019_04 AS presc
WHERE
  (bnf_code LIKE "0209000A0AA%BAB" ##dispersible 75mg
  OR bnf_code LIKE "0209000A0AA%JAJ" ##normal 75mg tabs
  OR bnf_code LIKE "0209000A0AA%KAK") ##enteric coated 75mg
'''

df_aspirin = bq.cached_read(sql, csv_path='aspirin_sevendays.csv')
df_aspirin.rename(columns={'PCO_Code':'pct'}, inplace=True)
df_aspirin.head(5)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,PCO_Name,pct,bnf_code,bnf_name,Items,Quantity
0,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,98,28
1,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,19,56
2,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,15,7
3,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,262,28
4,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,5,56


In [12]:
##check Englaish total 
eng_total = df_aspirin.sum()[['Items','Quantity']]
eng_total

Items       1830321
Quantity    3137207
dtype: object

In [13]:
##group and sum items
ccg_total = df_aspirin.groupby('PCO_Name').sum()[['Items']]
ccg_total.head(5)

,Items
PCO_Name,
"AIREDALE, WHARFEDALE AND CRAVEN CCG",5081
ASHFORD CCG,3521
BARKING & DAGENHAM CCG,6031
BARNET CCG,9132
BARNSLEY CCG,14304


In [14]:
##islolate prescriptions with quantities of 7
df_aspirin_sevendays = df_aspirin.loc[df_aspirin['Quantity'] == 7]
df_aspirin_sevendays.rename(columns={'Items':'sevendayitems'}, inplace=True) ##renaming column
df_aspirin_sevendays.head(5)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PCO_Name,pct,bnf_code,bnf_name,sevendayitems,Quantity
2,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,15,7
7,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,50,7
11,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,47,7
14,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,5,7
18,SOUTH WARWICKSHIRE CCG,05R00,0209000A0AAABAB,Aspirin Disper_Tab 75mg,80,7


In [15]:
ccg_sevenday_total = df_aspirin_sevendays.groupby('PCO_Name').sum()[['sevendayitems']]
ccg_sevenday_total.head(5)

,sevendayitems
PCO_Name,
"AIREDALE, WHARFEDALE AND CRAVEN CCG",600
ASHFORD CCG,119
BARKING & DAGENHAM CCG,2999
BARNET CCG,3235
BARNSLEY CCG,4509


In [16]:
dosette_boxes_ccg = pd.concat([ccg_total, ccg_sevenday_total], axis=1, join='inner')
dosette_boxes_ccg["percent_seven_days"] = 100*(dosette_boxes_ccg.sevendayitems/dosette_boxes_ccg.Items).fillna(0)
dosette_boxes_ccg.head()

,Items,sevendayitems,percent_seven_days
PCO_Name,,,
"AIREDALE, WHARFEDALE AND CRAVEN CCG",5081,600,11.808699
ASHFORD CCG,3521,119,3.379722
BARKING & DAGENHAM CCG,6031,2999,49.726414
BARNET CCG,9132,3235,35.424880
BARNSLEY CCG,14304,4509,31.522651


In [17]:
dosette_boxes_ccg.to_csv('aspirin.csv')

In [18]:
#work out english total
eng_total2 = dosette_boxes_ccg.sum()[['Items','sevendayitems']]
eng_total2

Items            1830222.0
sevendayitems     377700.0
dtype: float64

circa 21% of Aspirin 75mg items are written as a seven day prescription.